In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer

stopwords = set(nltk.corpus.stopwords.words('english'))

In [4]:
import json

In [6]:
json_data=open("anly610_amazon.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))

In [7]:
feed_titles = []
i = 0
for record in feeds_read_from_file:
#     record['id'] = i
#     print(record['id'], str(record['title']))
    i+=1
    feed_titles.append(str(record['title']))

In [10]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [37]:
def clstr_lda(num_topics, stories):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=0.05, min_df=0.001, max_features=200,
                                    tokenizer=tokenize_titles, ngram_range=(3,4))

    tf = tf_vectorizer.fit_transform(stories)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=300,
                                    learning_method='online', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [38]:
topics = clstr_lda(20, feed_titles)

Topic #0:
   |     | amazon   |  1 trillion |   amazon |  amazon  |  allege bezos phone |  allege bezos | allege bezos phone hack | allege bezos phone
Topic #1:
jeff bezos   |  need know | whatsapp security flaw |   need know | know whatsapp security | know whatsapp security flaw | need know whatsapp security | need know whatsapp |  need know whatsapp |   need
Topic #2:
spend   |  39 spend | 39 spend  |  39 spend  | 39 spend   | prime  39 | prime  39 spend |   0 | delivery   |   0 prime
Topic #3:
keep phone safe hackers | phone safe hackers | keep phone safe | amazon prime video | nt like jeff |   keep phone safe | nt like jeff bezos |    keep phone | bezos    keep | bezos   
Topic #4:
 1 billion |  giveaway  | amazon prime video | amazon   | saudi crown prince | crown prince hack |  1 trillion |  business insider | amazon great indian | bezos  hack
Topic #5:
 amazon  |  jeff bezos | new job india | jeff bezos phone | news  et |  business insider |   amazon | amazon prime video | jeff 